# 作业三：预训练语言模型计算PPL
姓名：陶瑞
学号：522030910024

In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

In [74]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("[Work on {}]".format(device))

[Work on cpu]


## 加载模型和Tokenizer

In [4]:
model_path = "./gpt2"
model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
model.eval()
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)

### Tokenizer

> 下面是一个例子，展示Tokenizer和模型的使用。理解下面的例子可能对你的大作业有帮助。

```python

Tokenizer会将句子分割成一个个token，然后将每个token转化为一个数字，这个数字就是这个token在词表中的id。

In [5]:
inputs = tokenizer("""GPT-2 is a transformers model pretrained on a very large corpus of English data in a self-supervised fashion.""", return_tensors="pt")
inputs

{'input_ids': tensor([[   38, 11571,    12,    17,   318,   257,  6121,   364,  2746,  2181,
         13363,   319,   257,   845,  1588, 35789,   286,  3594,  1366,   287,
           257,  2116,    12, 16668, 16149,  6977,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])}

可以将token id映射到对应的分词token

In [6]:
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
print(tokens)

['G', 'PT', '-', '2', 'Ġis', 'Ġa', 'Ġtransform', 'ers', 'Ġmodel', 'Ġpret', 'rained', 'Ġon', 'Ġa', 'Ġvery', 'Ġlarge', 'Ġcorpus', 'Ġof', 'ĠEnglish', 'Ġdata', 'Ġin', 'Ġa', 'Ġself', '-', 'super', 'vised', 'Ġfashion', '.']


可以使用`decode`方法将token id转化回原来的句子

In [7]:
decoded_string = tokenizer.decode([38, 11571, 12, 17, 318, 257, 6121, 364, 2746, 2181, 13363, 319, 257, 845, 1588, 35789, 286, 3594, 1366, 287, 257, 2116, 12, 16668, 16149, 6977, 13])
print(decoded_string)

GPT-2 is a transformers model pretrained on a very large corpus of English data in a self-supervised fashion.


### GPT2

GPT2是自回归式语言模型，可以根据前面的token预测下一个token。

将上面的token id输入到GPT2模型中，就可以得到每个token的概率分布

GPT2的输出的logits是一个三维张量，第一维是batch size，第二维是token的数量，第三维是词表的大小

> 注意：GPT2输出的是logits，需要经过softmax才能得到真正的概率分布

In [8]:
input_ids = inputs.input_ids.to(device)
with torch.no_grad():
    logits = model(input_ids).logits
print(logits.shape) # batch大小，序列长度，词表大小
print(logits[0, 0, :]) # 对于第一个词的预测logits，通过softmax后可以得到概率分布

torch.Size([1, 27, 50257])
tensor([-31.8241, -31.4346, -33.4860,  ..., -39.5281, -38.9088, -31.8361])


## 计算Perplexity (PPL)

PPL是语言模型的一个重要评价指标，表示模型对于给定的句子的概率分布的拟合程度。

计算公式为：
$$
PPL = \sqrt[N]{\prod_{i=1}^{N}\frac{1}{P(w_i|w_1,w_2,...,w_{i-1})}}
$$
通常可以转化为对数形式：
$$
PPL = \exp\left(\frac{1}{N}\sum_{i=1}^{N}-\log P(w_i|w_1,w_2,...,w_{i-1})\right)
$$

本节将实现GPT2模型的PPL计算

In [104]:
from torch.nn import Softmax, CrossEntropyLoss


def calculate_ppl(model, text):
    ## TODO: 首先将文本转换为输入token (7分)
    input_ids = tokenizer(text, return_tensors="pt")['input_ids'].to(model.device)

    # 获取模型的输出
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits
        labels = input_ids.to(logits.device)
        # GPT2每个位置都是预测下一个token的概率，所以需要将labels向左移动一位
        shift_logits = logits[..., :-1, :]      # 形状 (1, N, 50257) 很多负数
        shift_labels = labels[..., 1:]          # 形状 (1, N) 是在50257范围内的索引

        ## TODO: 根据logits和labels计算model在text上的ppl（8分）
        ## Hint: 可以直接通过Softmax获取概率值按照上面公式计算
        ## Hint2: 也可以尝试利用CrossEntropyLoss进行等价计算

        # 按照定义 使用softmax计算
        probs = Softmax(dim=-1)(shift_logits)   # 以 50257 维度做softmax 相当于 N 个归一化向量
        nll = -torch.log(torch.gather(probs, dim=-1, index=shift_labels.unsqueeze(-1)))
        mean_nll = nll.mean()
        ppl1 = torch.exp(mean_nll)

        # CrossEntropyLoss直接计算
        loss = CrossEntropyLoss(reduction='mean')(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        ppl2 = torch.exp(loss)

        assert torch.allclose(ppl1, ppl2), "Two methods not equal: {} vs {}".format(ppl1, ppl2)

    return ppl2.item()

print("Defined [calculate_ppl] function")

Defined [calculate_ppl] function


## 测试

In [105]:
text1 = "GPT-4 is a large multimodal model (accepting image and text inputs, emitting text outputs) that, while less capable than humans in many real-world scenarios, exhibits human-level performance on various professional and academic benchmarks."
text2 = "Until the rocket ship nearly imploded. On Nov. 17, OpenAI's nonprofit board of directors fired Altman, without warning or even much in the way of explanation. The surreal maneuvering that followed made the corporate dramas of Succession seem staid. Employees revolted. So did OpenAI's powerful investors; one even baselessly speculated that one of the directors who defenestrated Altman was a Chinese spy. The company's visionary chief scientist voted to oust his fellow co-founder, only to backtrack. Two interim CEOs came and went. The players postured via selfie, open letter, and heart emojis on social media. Meanwhile, the company's employees and its board of directors faced off in “a gigantic game of chicken,” says a person familiar with the discussions. At one point, OpenAI's whole staff threatened to quit if the board didn't resign and reinstall Altman within a few hours, three people involved in the standoff tell TIME. Then Altman looked set to decamp to Microsoft—with potentially hundreds of colleagues in tow. It seemed as if the company that catalyzed the AI boom might collapse overnight."
not_reasonable_text = "The quick brown fox jumps over the lazy dog, informing tiger behind it to fly away. Such scenrio was caught by bird hanging under table."
very_reasonable_text = "One plus one equals to two. Therefore, if we have an apple, and we have another apple, we will have two apples."
print(calculate_ppl(model, text1))
print(calculate_ppl(model, text2))
print("\nNot reasonable text:", calculate_ppl(model, not_reasonable_text))
print("Reasonabel text:", calculate_ppl(model, very_reasonable_text))

68.2021484375
46.45378112792969

Not reasonable text: 473.3460693359375
Reasonabel text: 20.868804931640625


## 实验总结

1. 实现细节
    * 将文本转换成输入 token，仿照前面的示例，用 GPT2 预训练模型的 `tokenize()` 直接编码 text，然后取其结果的 `['input_ids']` 项
    * 使用 softmax 按定义计算 PPL，首先 `Softmax(dim=-1)(shift_logits)` 转换成归一化的概率分布，然后 gather 成长度为 $N$ 的向量，对应 $P(w_i|w_1,w_2,...,w_{i-1})$，然后取对数，把乘积变成求和取平均。
    * 使用 CrossEntropyLoss 计算 PPL。按照交叉熵定义，每个位置上为 $-\log \frac{1}{N} P(w_i|w_1,w_2,...,w_{i-1})$，最后求和，提出公因子 $\frac{1}{N}$ 得到 $-\cfrac{1}{N}\sum\limits_{i=1}^{N}\log P(w_i|w_1,w_2,...,w_{i-1})$，这直接就是 PPL 的对数形式。

2. 实验结果
    除了给的两句话外，随机写了一句 `not_reasonable_text` 和 `very_reasonable_text`，前者使用不符合常理的动词名词搭配，描述了无法理解的现象（但符合语法），后者描述了一个简单推理逻辑，语法和例子都是常见的。由结果可以看出，前者的 PPL=473，远高于后者的 PPL=20.8

3. 其他观察
    下面对于 PPL 在什么句子下会更大/更小进行了一些额外的实验，我们对于句子主要进行以下几种指标的分类：
    * 语法正确/错误
    * 事实正确/不正确
    * 前后逻辑正确/不正确
    下面展示了其排列组合的八种句子，我们用<span style="color:red">“一加一等于二，所以如果我有一个苹果，又得到了一个苹果，那么我有两个苹果”</span>作为基准句子，进行【语法、事实、逻辑】三方面的改动。

实验结果总结如下表。可见影响 PPL 的主要因素是【语法】，其他因素相比而言影响微乎其微。除语法之外，【逻辑】相比【事实】而言更重要一些。

| 语法 | 事实 | 逻辑 | PPL |
| :-: | :-: | :-: | :-: |
| ❌ | ❌ | ❌ | 309.19 |
| ✔ | ❌ | ❌ | 25.89 |
| ❌ | ✔ | ❌ | 283.29 |
| ✔ | ✔ | ❌ | 26.38 |
| ❌ | ❌ | ✔ | 277.41 |
| ✔ | ❌ | ✔ | 21.79 |
| ❌ | ✔ | ✔ | 249.41 |
| ✔ | ✔ | ✔ | 20.87 |

In [106]:
# 三个位置的数字分别表示 [语法, 事实, 逻辑]
_000 = "One plus one is ten. Therefore, has we an apple if, and another three we have, five we will have apples."   # 语法错误, 1+1=10; 1+3=5
_100 = "One plus one equals to ten. Therefore, if we have an apple, and we have another three apples, we will have five apples."   # 语法正确, 1+1=10; 1+3=5
_010 = "One plus one is two. Therefore, has we an apple if, and another three we have, four we will have apples."   # 语法错误, 1+1=2; 1+3=4
_110 = "One plus one equals to ten. Therefore, if we have an apple, and we have another three apples, we will have four apples."   # 语法正确, 1+1=2; 1+3=4
_001 = "One plus one is ten. Therefore, has we an apple if, and another we have, ten we will have apples."   # 语法错误, 1+1=10; 1+1=10
_101 = "One plus one equals to ten. Therefore, if we have an apple, and we have another apple, we will have ten apples."   # 语法正确, 1+1=2; 1+1=10
_011 = "One plus one is two. Therefore, has we an apple if, and another we have, two we will have apples."   # 语法错误, 1+1=2; 1+1=2
_111 = "One plus one equals to two. Therefore, if we have an apple, and we have another apple, we will have two apples."   # 语法正确, 1+1=2; 1+1=2

sentences = [_000, _100, _010, _110, _001, _101, _011, _111]
for i, sentence in enumerate(sentences):
    print("{} | {}".format(i, calculate_ppl(model, sentence)))

0 | 309.18878173828125
1 | 25.887935638427734
2 | 283.2857971191406
3 | 26.37828826904297
4 | 277.4051818847656
5 | 21.7869815826416
6 | 249.40921020507812
7 | 20.868804931640625
